In [ ]:
%%capture

!pip install guardrails-ai
!pip install langchain
!pip install --upgrade python-dotenv

In [ ]:
import openai
import os
from dotenv import load_dotenv

from pydantic import BaseModel
from rich import print
from typing import List

import guardrails as gd

load_dotenv('key.env')

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

##Using Open AI

In [ ]:
class Movie(BaseModel):
    rank: int
    title: str
    details: str


class Director(BaseModel):
    """A movie director"""

    name: str
    movies: List[Movie]

In [ ]:
guard = gd.Guard.from_pydantic(Director, prompt="Generate data about a movie director.")

raw_llm_output, validated_output = guard(
    openai.ChatCompletion.create,
    model="gpt-3.5-turbo",
    max_tokens=1024,
    temperature=0.0,
)

/usr/local/lib/python3.10/dist-packages/guardrails/prompt/prompt.py:23: UserWarning: Prompt does not have any variables, if you are migrating follow the new variable convention documented here: https://docs.getguardrails.ai/0-2-migration/
  warnings.warn(


In [ ]:
type(raw_llm_output)

str

In [ ]:
print(type(validated_output))

<class 'dict'>

##Using Langchain

In [ ]:
rail_spec = """
<rail version="0.1">
<output>
<object name="patient_info">
<string description="Patient's gender" name="gender"></string>
<integer format="valid-range: 0 100" name="age"></integer>
<string description="Symptoms that the patient is currently experiencing" name="symptoms"></string>
</object>
</output>

<prompt>

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
</prompt>
</rail>
"""

In [ ]:
from rich import print

from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [ ]:
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=openai.ChatCompletion.create)

In [ ]:
print(output_parser.guard.prompt)

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's 
information.

${doctors_notes}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <object name="patient_info">
        <string description="Patient's gender" name="gender"/>
        <integer format="valid-range: min=0 max=100" name="age"/>
        <string description="Symptoms that the patient is currently experiencing" name="symptoms"/>
    </object>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [ ]:
prompt = PromptTemplate(
    template=output_parser.guard.prompt.escape(),
    input_variables=output_parser.guard.prompt.variable_names,
)

In [ ]:
model = OpenAI(temperature=0)


doctors_notes = """
49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream
"""
output = model(prompt.format_prompt(doctors_notes=doctors_notes).to_string())

In [ ]:
print(output_parser.parse(output))

{
    'patient_info': {
        'gender': 'Male',
        'age': 49,
        'symptoms': 'Chronic macular rash to face & hair, worse in beard, eyebrows & nares. Itchy, flaky, slightly 
scaly. Moderate response to OTC steroid cream'
    }
}